# Лабораторная работа №4 (Проведение исследований со случайным лесом)

## Создание бейзлайна и оценка качества

### Обучение моделей из sklearn для выбранных наборов данных

#### Импорт библиотек

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer
from scipy.spatial import distance
import numpy as np
from collections import Counter

#### Подготовка данных

##### Классификация (Rice MSC Dataset)

**Загрузка данных:**

In [4]:
rice_data = pd.read_csv('Rice_MSC_Dataset.csv')

X_rice = rice_data.iloc[:, :-1]  # Признаки
y_rice = rice_data.iloc[:, -1]   # Метки классов

label_encoder = LabelEncoder()
y_rice_encoded = label_encoder.fit_transform(y_rice)

**Разделение на тренировочный и тестовый наборы:**

In [5]:
X_train_rice, X_test_rice, y_train_rice, y_test_rice = train_test_split(X_rice, y_rice_encoded, test_size=0.2, random_state=42)

**Замена пропущенных значений**

In [6]:
def simple_imputer_df(x_train, x_test):
    imputer = SimpleImputer(strategy='mean')
    return pd.DataFrame(imputer.fit_transform(x_train)), pd.DataFrame(imputer.transform(x_test))

In [7]:
X_train_rice, X_test_rice = simple_imputer_df(X_train_rice, X_test_rice)

**Масштабирование данных**

In [8]:
def scaling(x_train, x_test):
    scaler = StandardScaler()
    return scaler.fit_transform(x_train), scaler.transform(x_test)

In [9]:
X_train_rice, X_test_rice = scaling(X_train_rice, X_test_rice)

##### Регрессия (Gold Price Regression)

**Загрузка данных:**

In [128]:
gold_data = pd.read_csv('financial_regression.csv')

X_gold = gold_data.iloc[:, 1:-1]  # Признаки
y_gold = gold_data.iloc[:, -1]   # Целевая переменная

**Разделение на тренировочный и тестовый наборы:**

In [11]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold, y_gold, test_size=0.2, random_state=42)

**Замена пропущенных значений:**

In [12]:
def simple_imputer(x_train, x_test):
    imputer = SimpleImputer(strategy='mean')
    return imputer.fit_transform(x_train), imputer.transform(x_test)

In [13]:
X_train_gold, X_test_gold = simple_imputer(X_train_gold, X_test_gold)

**Удаление строк с пропущенными значениями:**

In [14]:
def delete_none(x_train, y_train, x_test, y_test):
    mask = ~np.isnan(y_train)
    mask_test = ~np.isnan(y_test)
    return x_train[mask], y_train[mask], x_test[mask_test], y_test[mask_test]

In [15]:
X_train_gold, y_train_gold, X_test_gold, y_test_gold = delete_none(X_train_gold, y_train_gold, X_test_gold, y_test_gold)

#### Обучение моделей

##### Классификация

In [17]:
classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_train_rice, y_train_rice)

RandomForestClassifier(random_state=42)

##### Регрессия

In [18]:
regressor = RandomForestRegressor(random_state=42)
regressor.fit(X_train_gold, y_train_gold)

RandomForestRegressor(random_state=42)

### Оценка качества моделей по выбранным метрикам

##### Классификация

In [19]:
def quality_eval_classification(X_test, y_test, model):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1}")c
    return accuracy, f1

In [21]:
accuracy_origin, f1_origin = quality_eval_classification(X_test_rice, y_test_rice, classifier)

Accuracy: 0.9986666666666667
F1 Score: 0.9986666534740459


##### Регрессия

In [22]:
def quality_eval_regression(X_test, y_test, model):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Mean Absolute Error: {mae}")
    print(f"R-squared: {r2}")
    return mae, r2

In [23]:
mae_origin, r2_origin = quality_eval_regression(X_test_gold, y_test_gold, regressor)

Mean Absolute Error: 2444766.2500400534
R-squared: 0.610479164922296


## Улучшение бейзлайна

### Гипотезы

1. Подбор гиперпараметров:
   - Использование Grid Search с кросс-валидацией для оптимизации модели.

### Проверка гипотез

#### 1. Подбор гиперпараметров

##### Классификация

Подготовка данных:

In [25]:
X_train_rice, X_test_rice, y_train_rice, y_test_rice = train_test_split(X_rice, y_rice_encoded, test_size=0.2, random_state=42)
X_train_rice, X_test_rice = simple_imputer_df(X_train_rice, X_test_rice)
X_train_rice, X_test_rice = scaling(X_train_rice, X_test_rice)

Определение параметров для перебора:

In [26]:
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

Настройка и обучение GridSearchCV:

In [27]:
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train_rice, y_train_rice)

GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 150]})

Вывод лучших параметров и результата:

In [28]:
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)

Best parameters found:  {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 150}
Best cross-validation accuracy:  0.9986833333333335


Оценка качества модели:

In [29]:
quality_eval_classification(X_test_rice, y_test_rice, grid_search.best_estimator_)

Accuracy: 0.9987333333333334
F1 Score: 0.9987333097767334


(0.9987333333333334, 0.9987333097767334)

##### Регрессия

Подготовка данных:

In [30]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold, y_gold, test_size=0.2, random_state=42)
X_train_gold, X_test_gold = simple_imputer(X_train_gold, X_test_gold)
X_train_gold, X_test_gold = scaling(X_train_gold, X_test_gold)
X_train_gold, y_train_gold, X_test_gold, y_test_gold = delete_none(X_train_gold, y_train_gold, X_test_gold, y_test_gold)

Определение параметров для перебора:

In [31]:
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

Настройка и обучение GridSearchCV:

In [32]:
grid_search = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train_gold, y_train_gold)

/home/aprold/study/MAI-MMTT/venv/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
243 fits failed out of a total of 729.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
79 fits failed with the following error:
Traceback (most recent call last):
  File "/home/aprold/study/MAI-MMTT/venv/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/aprold/study/MAI-MMTT/venv/lib/python3.12/site-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/home/aprold/study/MAI-MMTT/venv/lib/python3.12/site-packages/sklearn/base.py", line 436, in _validate_params
    valid

GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 150]})

Вывод лучших параметров и результата:

In [33]:
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)

Best parameters found:  {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 150}
Best cross-validation accuracy:  0.6640723937986626


Оценка модели с лучшими параметрами на тестовом наборе:

In [34]:
quality_eval_regression(X_test_gold, y_test_gold, grid_search.best_estimator_)

Mean Absolute Error: 2392735.5308633246
R-squared: 0.6161405634184933


(2392735.5308633246, 0.6161405634184933)

### Формирование улучшенного бейзлайна

##### Классификация

Формирование улучшенного бейзлайна:

In [35]:
X_train_rice, X_test_rice, y_train_rice, y_test_rice = train_test_split(X_rice, y_rice_encoded, test_size=0.2, random_state=42)
X_train_rice, X_test_rice = simple_imputer_df(X_train_rice, X_test_rice)
X_train_rice, X_test_rice = scaling(X_train_rice, X_test_rice)
                                   
classifier = RandomForestClassifier(random_state=42, max_depth=None, min_samples_split=2, n_estimators=150)

##### Регрессия

In [37]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold, y_gold, test_size=0.2, random_state=42)
X_train_gold, X_test_gold = simple_imputer(X_train_gold, X_test_gold)
X_train_gold, X_test_gold = scaling(X_train_gold, X_test_gold)
X_train_gold, y_train_gold, X_test_gold, y_test_gold = delete_none(X_train_gold, y_train_gold, X_test_gold, y_test_gold)

regressor = RandomForestRegressor(random_state=42, max_depth=20, max_features="sqrt", min_samples_leaf=1, min_samples_split=2, n_estimators=150)

### Обучение модели с улучшенным бейзлайном

##### Классификация

Обучение модели:

In [38]:
classifier.fit(X_train_rice, y_train_rice)

RandomForestClassifier(n_estimators=150, random_state=42)

##### Регрессия

Обучение модели:

In [39]:
regressor.fit(X_train_gold, y_train_gold)

RandomForestRegressor(max_depth=20, max_features='sqrt', n_estimators=150,
                      random_state=42)

### Оценка качества модели с улучшенным бейзлайном

##### Классификация

Оценка качества модели:

In [40]:
accuracy_improved, f1_improved = quality_eval_classification(X_test_rice, y_test_rice, classifier)

Accuracy: 0.9987333333333334
F1 Score: 0.9987333097767334


##### Регрессия

Оценка качества модели:

In [41]:
mae_improved, r2_improved = quality_eval_regression(X_test_gold, y_test_gold, regressor)

Mean Absolute Error: 2392735.5308633246
R-squared: 0.6161405634184933


### Сравнение результатов первоначального бейзлайна с улучшенным бейзлайном

##### Классификация

Видим, что результаты почти идентичны:

In [42]:
print(f"Accuracy difference: {accuracy_improved-accuracy_origin}")
print(f"F1 Score difference: {f1_improved-f1_origin}")

Accuracy difference: 6.666666666665932e-05
F1 Score difference: 6.665630268742895e-05


##### Регрессия

Видим, что результаты немного улучшились:

In [43]:
print(f"Mean Absolute Error difference: {mae_origin-mae_improved}")
print(f"R-squared difference: {r2_improved-r2_origin}")

Mean Absolute Error difference: 52030.719176728744
R-squared difference: 0.00566139849619729


### Выводы

Улучшенный бейзлайн оказался лучше базового, хотя и не так сильно. Скорей всего, нужно выдвигать и проверять другие гипотезы.

## Имплементация алгоритма машинного обучения

### Самостоятельная имплементация

Случайный лес (Random Forest) — это ансамблевый алгоритм машинного обучения, использующий метод объединения множества базовых моделей (в данном случае деревьев решений) для улучшения общей производительности. 

##### Классификация

Сначала реализауем дерево решений

In [134]:
class DecisionTreeClassifierCustom:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def fit(self, X, y):
        self.n_classes_ = len(set(y))
        self.n_features_ = X.shape[1]
        self.tree_ = self._grow_tree(X, y)

    def _gini(self, y):
        m = len(y)
        return 1.0 - sum((np.sum(y == c) / m) ** 2 for c in np.unique(y))

    def _best_split(self, X, y):
        # Placeholder for storing best parameters
        best_idx, best_thr = None, None
        best_gini = 1.0
        
        for idx in range(self.n_features_):
            thresholds, classes = zip(*sorted(zip(X[:, idx], y)))
            num_left = [0] * self.n_classes_
            num_right = [np.sum(y == c) for c in range(self.n_classes_)]
            
            for i in range(1, len(y)):
                c = classes[i - 1]
                num_left[c] += 1
                num_right[c] -= 1
                
                gini_left = 1.0 - sum((num_left[x] / i) ** 2 for x in range(self.n_classes_))
                gini_right = 1.0 - sum((num_right[x] / (len(y) - i)) ** 2 for x in range(self.n_classes_))
                
                gini = (i * gini_left + (len(y) - i) * gini_right) / len(y)
                
                if thresholds[i] == thresholds[i - 1]:
                    continue
                
                if gini < best_gini:
                    best_gini = gini
                    best_idx = idx
                    best_thr = (thresholds[i] + thresholds[i - 1]) / 2
        
        return best_idx, best_thr

    def _grow_tree(self, X, y, depth=0):
        num_samples_per_class = [np.sum(y == i) for i in range(self.n_classes_)]
        predicted_class = np.argmax(num_samples_per_class)
        node = NodeClassifierCustom(
            gini=self._gini(y),
            num_samples=len(y),
            num_samples_per_class=num_samples_per_class,
            predicted_class=predicted_class
        )

        if depth < self.max_depth:
            idx, thr = self._best_split(X, y)
            if idx is not None:
                indices_left = X[:, idx] < thr
                X_left, y_left = X[indices_left], y[indices_left]
                X_right, y_right = X[~indices_left], y[~indices_left]
                node.feature_index = idx
                node.threshold = thr
                node.left = self._grow_tree(X_left, y_left, depth + 1)
                node.right = self._grow_tree(X_right, y_right, depth + 1)
        return node

    def _predict(self, inputs):
        node = self.tree_
        while node.left:
            if inputs[node.feature_index] < node.threshold:
                node = node.left
            else:
                node = node.right
        return node.predicted_class

    def predict(self, X):
        return [self._predict(inputs) for inputs in X]
        
class NodeClassifierCustom:
    def __init__(self, gini, num_samples, num_samples_per_class, predicted_class):
        self.gini = gini
        self.num_samples = num_samples
        self.num_samples_per_class = num_samples_per_class
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0 
        self.left = None
        self.right = None

Теперь используем наше дерево решений для создания ансамбля — случайного леса:

In [135]:
class RandomForestClassifierCustom:
    def __init__(self, n_trees=10, max_depth=None, n_features=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.n_features = n_features
        self.trees = []

    def _bootstrap_sample(self, X, y):
        n_samples = X.shape[0]
        indices = np.random.choice(n_samples, n_samples, replace=True)
        return X[indices], y[indices]

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTreeClassifierCustom(max_depth=self.max_depth)
            X_sample, y_sample = self._bootstrap_sample(X, y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(tree_preds, 0, 1)
        return [Counter(tree_pred).most_common(1)[0][0] for tree_pred in tree_preds]

##### Регрессия

Сначала реализауем дерево решений

In [169]:
class DecisionTreeRegressorCustom:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def fit(self, X, y):
        self.n_features_ = X.shape[1]
        self.tree_ = self._grow_tree(X, y)

    def _mse(self, y):
        if len(y) == 0:
            return 0
        return np.mean((y - np.mean(y)) ** 2)

    def _best_split(self, X, y):
        best_idx, best_thr = None, None
        best_mse = np.inf

        for idx in range(self.n_features_):
            thresholds, values = zip(*sorted(zip(X[:, idx], y)))
            mse_left = mse_right = 0
            sum_left = sum_right = 0
            n_left = n_right = 0

            for i in range(1, len(y)):
                sum_left += values[i - 1]
                mse_left = np.mean((values[:i] - np.mean(values[:i])) ** 2) if i > 0 else 0

                sum_right -= values[i - 1]
                mse_right = np.mean((values[i:] - np.mean(values[i:])) ** 2) if i < len(y) else 0
                
                mse = (i * mse_left + (len(y) - i) * mse_right) / len(y)

                if thresholds[i] == thresholds[i - 1]:
                    continue

                if mse < best_mse:
                    best_mse = mse
                    best_idx = idx
                    best_thr = (thresholds[i] + thresholds[i - 1]) / 2
        
        return best_idx, best_thr

    def _grow_tree(self, X, y, depth=0):
        node_value = np.mean(y)
        node = NodeRegressorCustom(
            num_samples=len(y),
            node_value=node_value
        )

        if depth < self.max_depth:
            idx, thr = self._best_split(X, y)
            if idx is not None:
                indices_left = X[:, idx] < thr
                X_left, y_left = X[indices_left], y[indices_left]
                X_right, y_right = X[~indices_left], y[~indices_left]
                node.feature_index = idx
                node.threshold = thr
                node.left = self._grow_tree(X_left, y_left, depth + 1)
                node.right = self._grow_tree(X_right, y_right, depth + 1)
        return node

    def _predict(self, inputs):
        node = self.tree_
        while node.left:
            if inputs[node.feature_index] < node.threshold:
                node = node.left
            else:
                node = node.right
        return node.node_value

    def predict(self, X):
        return [self._predict(inputs) for inputs in X]

class NodeRegressorCustom:
    def __init__(self, num_samples, node_value):
        self.num_samples = num_samples
        self.node_value = node_value
        self.feature_index = 0
        self.threshold = 0 
        self.left = None
        self.right = None

Теперь используем наше дерево решений для создания ансамбля — случайного леса:

In [170]:
class RandomForestRegressorCustom:
    def __init__(self, n_trees=10, max_depth=None, n_features=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.n_features = n_features
        self.trees = []

    def _bootstrap_sample(self, X, y):
        n_samples = X.shape[0]
        indices = np.random.choice(n_samples, n_samples, replace=True)
        return X[indices], y[indices]

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTreeRegressorCustom(max_depth=self.max_depth)
            X_sample, y_sample = self._bootstrap_sample(X, y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        return np.mean(tree_preds, axis=0)

### Обучение имплементированной модели

##### Классификация

Подготовка данных:

In [177]:
X_train_rice, X_test_rice, y_train_rice, y_test_rice = train_test_split(X_rice, y_rice_encoded, test_size=0.2, random_state=42)
X_train_rice, X_test_rice = simple_imputer_df(X_train_rice, X_test_rice)
X_train_rice, X_test_rice = scaling(X_train_rice, X_test_rice)

classifier = RandomForestClassifierCustom(max_depth=5)

Обучение модели:

In [178]:
classifier.fit(X_train_rice, y_train_rice)

##### Регрессия

Подготовка данных:

In [179]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold, y_gold, test_size=0.2, random_state=42)
X_train_gold, X_test_gold = simple_imputer(X_train_gold, X_test_gold)
X_train_gold, X_test_gold = scaling(X_train_gold, X_test_gold)
y_train_gold = y_train_gold.to_numpy()
y_test_gold = y_test_gold.to_numpy()
X_train_gold, y_train_gold, X_test_gold, y_test_gold = delete_none(X_train_gold, y_train_gold, X_test_gold, y_test_gold)

regressor = RandomForestRegressorCustom(max_depth=5)

Обучение модели:

In [180]:
regressor.fit(X_train_gold, y_train_gold)

### Оценка качества имплементированных моделей

##### Классификация

Оценка качества модели:

In [185]:
accuracy_my, f1_my = quality_eval_classification(X_test_rice, y_test_rice, classifier)

Accuracy: 0.9938
F1 Score: 0.9938038533389124


##### Регрессия

Оценка качества модели:

In [186]:
mae_my, r2_my = quality_eval_regression(X_test_gold, y_test_gold, regressor)

Mean Absolute Error: 2834959.4890514747
R-squared: 0.5206168695894298


### Сравнение результатов первоначального бейзлайна и имплементированных моделей

##### Классификация

Видим, что результаты имплементированной модели немного хуже:

In [189]:
print(f"Accuracy difference: {accuracy_my-accuracy_origin}")
print(f"F1 Score difference: {f1_my-f1_origin}")

Accuracy difference: -0.004866666666666686
F1 Score difference: -0.004862800135133516


##### Регрессия

Видим, что результаты имплементированной модели хуже:

In [190]:
print(f"Mean Absolute Error difference: {mae_my-mae_origin}")
print(f"R-squared difference: {r2_my-r2_origin}")

Mean Absolute Error difference: 390193.23901142133
R-squared difference: -0.08986229533286627


### Выводы

Оценка качества наших имплементированных моделей оказалась хуже.
Можно сделать следующие выводы:
1. Предобработка данных важна: Масштабирование и нормализация данных могут значительно повлиять на качество модели.
2. Оптимизация гиперпараметров: Правильная настройка параметров, таких как темп обучения и количество итераций, необходима для достижения оптимальной производительности.
3. Регуляризация: Помогает избежать переобучения и может улучшить обобщающую способность модели.
4. Алгоритмическая точность: Правильная реализация алгоритма и внимание к численным ошибкам критичны для качества модели.
5. Используйте проверенные решения: Изучение и сравнение с библиотечными реализациями может помочь выявить слабые места вашей модели.
6. Тщательно изучайте данные: Соблюдение чистоты, отсутствие пропусков и ошибок в данных — важный аспект построения качественной модели.

### Добавление техник из улучшенного бейзлайна

##### Классификация

In [191]:
X_train_rice, X_test_rice, y_train_rice, y_test_rice = train_test_split(X_rice, y_rice_encoded, test_size=0.2, random_state=42)
X_train_rice, X_test_rice = simple_imputer_df(X_train_rice, X_test_rice)
X_train_rice, X_test_rice = scaling(X_train_rice, X_test_rice)

classifier = RandomForestClassifierCustom(max_depth=5, n_trees=20)

##### Регрессия

Формирование улучшенного бейзлайна:

In [192]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold, y_gold, test_size=0.2, random_state=42)
X_train_gold, X_test_gold = simple_imputer(X_train_gold, X_test_gold)
X_train_gold, X_test_gold = scaling(X_train_gold, X_test_gold)
y_train_gold = y_train_gold.to_numpy()
y_test_gold = y_test_gold.to_numpy()
X_train_gold, y_train_gold, X_test_gold, y_test_gold = delete_none(X_train_gold, y_train_gold, X_test_gold, y_test_gold)

regressor = RandomForestRegressorCustom(max_depth=20, n_trees=10)

### Обучение на улучшенном бейзлайне

##### Классификация

In [193]:
classifier.fit(X_train_rice, y_train_rice)

##### Регрессия

In [194]:
regressor.fit(X_train_gold, y_train_gold)

### Оценка качества моделей на улучшенном бейзлайне

##### Классификация

In [196]:
accuracy_my_improved, f1_my_improved = quality_eval_classification(X_test_rice, y_test_rice, classifier)

Accuracy: 0.9944666666666667
F1 Score: 0.9944704125297763


##### Регрессия

In [197]:
mae_my_impoved, r2_my_impoved = quality_eval_regression(X_test_gold, y_test_gold, regressor)

Mean Absolute Error: 2570320.2829245282
R-squared: 0.5796125932674412


### Сравнение результатов

##### Классификация

In [198]:
print(f"Accuracy difference: {accuracy_my_improved-accuracy_improved}")
print(f"F1 Score difference: {f1_my_improved-f1_improved}")

Accuracy difference: -0.004266666666666641
F1 Score difference: -0.004262897246957054


##### Регрессия

In [199]:
print(f"Mean Absolute Error difference: {mae_my_impoved-mae_improved}")
print(f"R-squared difference: {r2_my_impoved-r2_improved}")

Mean Absolute Error difference: 177584.7520612036
R-squared difference: -0.036527970151052136


### Выводы

Улучшенный бейзлайн собственной имплементации получился немного хуже улучшенного бейзлайна готовой реализации. Вероятно, это из-за того, что было выбрано меньше итераций для обучения модели собственной имплементации.